# Scraping Similar Vehicles

For the third model we want to be able to show the user information about vehicles that are considered similar to the one they are looking at. The Craiglist dataset does not have that kind of information within it and it is certainly not possible to cluster to get that information. Instead we will find a website which already presents that information and create our own similar vehicle dataset.

In [10]:
import pandas as pd
from pathlib import Path
import sys
sys.path.append("..")
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import urllib.request
from car_purchase_help.data_processing import format_raw_df
from collections import defaultdict

In [2]:
# https://www.kaggle.com/austinreese/craigslist-carstrucks-data/
df = pd.read_csv(Path('../data/vehicles.csv'))
df = format_raw_df(df)

First we will try to find similar cars for one make and model. Let's use a common car to begin with

In [3]:
test_manu = 'honda'
test_model = 'accord'

An Australian website was found to have good similar car information with a url which is easy to generate. The HTML containing the similar car information appears to be accessible enough for Beautiful Soup to find the necessary information.

In [ ]:
url = f"https://www.carsguide.com.au/{test_manu}/{test_model}/vs"

The website automatically blocks access appearing to come from Python so we give user information in the header of the request to make it appear like a browser is trying to access it.

In [ ]:
user_agent = "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-GB; rv:1.8.1.6) Gecko/20070725 Firefox/2.0.0.6"
headers = {'User-Agent':user_agent} 

In [4]:
request = urllib.request.Request(url, None, headers)
response = urllib.request.urlopen(request)
data = response.read()

In [5]:
test_soup = BeautifulSoup(data, 'html.parser')

We can see below that we successfully got the HTML from the site. Our information is contained within it.

In [6]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>     <html class="carsguide no-details no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>        <html class="carsguide no-details no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>        <html class="carsguide no-details lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="carsguide no-details">
 <!--<![endif]-->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={xpid:"VQQAVVVTGwIIVVRUAAAHVw==",licenseKey:"5ab5008507",applicationID:"181567101"};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s

We use Beautiful Soups find_all method to find the tags we are interested in. We will loop over then extracting the car information from the hyperlink references.

In [12]:
test_similar_cars = set()
for car in soup.find_all("div", "col-lg-2 col-sm-3 col-md-3 col-xs-6 item"):
    for link in car.find_all("a", href=True):
        test_similar_cars.add(link['href'].split('/')[-1])

In [13]:
test_similar_cars

{'alfa-romeo-stelvio',
 'audi-q5',
 'audi-q7',
 'audi-sq5',
 'audi-sq7',
 'bentley-bentayga',
 'bmw-x3',
 'bmw-x4',
 'bmw-x5',
 'bmw-x6',
 'ford-everest',
 'hyundai-santa-fe',
 'infiniti-qx80',
 'jaguar-e-pace',
 'jaguar-f-pace',
 'jeep-grand-cherokee',
 'land-rover-discovery-sport',
 'land-rover-range-rover',
 'land-rover-range-rover-evoque',
 'land-rover-range-rover-sport',
 'lexus-lx',
 'lexus-nx',
 'lexus-rx',
 'maserati-levante',
 'mazda-cx-8',
 'mazda-cx-9',
 'mercedes-benz-gla-class',
 'mercedes-benz-glc-class',
 'mercedes-benz-gle-class',
 'mercedes-benz-gls-class',
 'mitsubishi-pajero',
 'nissan-pathfinder',
 'nissan-patrol',
 'porsche-cayenne',
 'porsche-macan',
 'tesla-model-x',
 'toyota-fortuner',
 'toyota-kluger',
 'toyota-landcruiser',
 'toyota-landcruiser-prado',
 'volkswagen-touareg',
 'volvo-xc60',
 'volvo-xc90'}

Our test above was successful so now we can loop over all manufacturers and models in our dataset attempting to save all the similar vehicles. We save our results in a default dictionary of sets.

In [8]:
similar_cars = defaultdict(set)
for manu in tqdm(df['manufacturer'].unique()):
    df_manu = df[df['manufacturer'] == manu]
    for model in tqdm(df_manu['model'].unique()):
        # Many of these urls will be invalid. We replaces spaces with dashes in an attempt to get more
        url = f"https://www.carsguide.com.au/{manu}/{model}/vs".replace(" ", "-")
        try:
            request = urllib.request.Request(url, None, headers)
            response = urllib.request.urlopen(request)
            data = response.read()
            soup = BeautifulSoup(data, 'html.parser')
            for car in soup.find_all("div", "col-lg-2 col-sm-3 col-md-3 col-xs-6 item"):
                for link in car.find_all("a", href=True):
                    similar_cars[f"{manu},{model}"].add(link['href'].split('/')[-1])
        except Exception as e:
            print(f"Failed for {manu}, {model} with error {e}")

Failed for acura, tl with error HTTP Error 404: Not Found
Failed for acura, mdx with error HTTP Error 404: Not Found
Failed for acura, nan with error HTTP Error 404: Not Found
Failed for acura, tsx with error HTTP Error 404: Not Found
Failed for acura, mdx sport utility 4d with error HTTP Error 404: Not Found
Failed for acura, tl 5-speed at with error HTTP Error 404: Not Found
Failed for acura, rdx with error HTTP Error 404: Not Found
Failed for acura, mdx tech package with error HTTP Error 404: Not Found
Failed for acura, rsx type s 2dr hatchback with error HTTP Error 404: Not Found
Failed for acura, tlx with error HTTP Error 404: Not Found
Failed for acura, mdx touring 4wd with error HTTP Error 404: Not Found
Failed for acura, 3.2 tl with error HTTP Error 404: Not Found
Failed for acura, integra with error HTTP Error 404: Not Found
Failed for acura, ilx with error HTTP Error 404: Not Found
Failed for acura, tl tech package with error HTTP Error 404: Not Found
Failed for acura, mdx sh

Failed for acura, mdx 3.5l with error HTTP Error 404: Not Found
Failed for acura, tl - v6 fully loaded!!! - with error HTTP Error 404: Not Found
Failed for acura, tl type s with error HTTP Error 404: Not Found
Failed for acura, zdx technology awd with error HTTP Error 404: Not Found
Failed for acura, mdx touring 4x4 with error HTTP Error 404: Not Found
Failed for acura, 3.2 tl s with error HTTP Error 404: Not Found
Failed for acura, cl 3.0 with error HTTP Error 404: Not Found
Failed for acura, tlx 4dr sdn fwd tech with error HTTP Error 404: Not Found
Failed for acura, tl w/navi with error HTTP Error 404: Not Found
Failed for acura, mdx w/advance pkg with error HTTP Error 404: Not Found
Failed for acura, mdx touring w/navi w/res with error HTTP Error 404: Not Found
Failed for acura, tl w/tech with error HTTP Error 404: Not Found
Failed for acura, mdx sh-awd w tech with error HTTP Error 404: Not Found
Failed for acura, tl w/advance with error HTTP Error 404: Not Found
Failed for acura, t

Failed for acura, rdx advance with error HTTP Error 404: Not Found
Failed for acura, vigor with error HTTP Error 404: Not Found
Failed for acura, mdx sh-awd technology with error HTTP Error 404: Not Found
Failed for acura, tlx 2.4 sedan with error HTTP Error 404: Not Found
Failed for acura, tlx technology fwd with error HTTP Error 404: Not Found
Failed for acura, rdx technology 4x4 with error HTTP Error 404: Not Found
Failed for acura, tsx tech pkg v6 with error HTTP Error 404: Not Found
Failed for acura, rl sh-awd w/tech with error <urlopen error [Errno 0] Error>
Failed for acura, mdx advance with error HTTP Error 404: Not Found
Failed for acura, integra gsr with error HTTP Error 404: Not Found
Failed for acura, ilx technology with error HTTP Error 404: Not Found
Failed for acura, rdx advance pkg with error HTTP Error 404: Not Found
Failed for acura, tsx se with error HTTP Error 404: Not Found
Failed for acura, tl technology fwd with error HTTP Error 404: Not Found
Failed for acura, 2

Failed for jaguar, xf premium with error HTTP Error 404: Not Found
Failed for jaguar, xk coupe 2d with error HTTP Error 404: Not Found
Failed for jaguar, xk with error HTTP Error 404: Not Found
Failed for jaguar, x-type with error HTTP Error 404: Not Found
Failed for jaguar, xk8 convertible with error HTTP Error 404: Not Found
Failed for jaguar, xj8 with error HTTP Error 404: Not Found
Failed for jaguar, xjl supercharged with error HTTP Error 404: Not Found
Failed for jaguar, xk8 with error HTTP Error 404: Not Found
Failed for jaguar, s-type with error HTTP Error 404: Not Found
Failed for jaguar, xf portfolio with error HTTP Error 404: Not Found
Failed for jaguar, s type 4.0 with error HTTP Error 404: Not Found
Failed for jaguar, xjr with error HTTP Error 404: Not Found
Failed for jaguar, xj6 with error HTTP Error 404: Not Found
Failed for jaguar, xjl portfolio with error HTTP Error 404: Not Found
Failed for jaguar, xj-series with error HTTP Error 404: Not Found
Failed for jaguar, xf p

Failed for honda, element with error HTTP Error 404: Not Found
Failed for honda, accord lx-s coupe 2d with error HTTP Error 404: Not Found
Failed for honda, civic lx with error HTTP Error 404: Not Found
Failed for honda, pilot with error HTTP Error 404: Not Found
Failed for honda, insight with error HTTP Error 404: Not Found
Failed for honda, element sc with error HTTP Error 404: Not Found
Failed for honda, accord coupe with error HTTP Error 404: Not Found
Failed for honda, accord exl with error HTTP Error 404: Not Found
Failed for honda, civic lx-p with error HTTP Error 404: Not Found
Failed for honda, civic si coupe 2d with error HTTP Error 404: Not Found
Failed for honda, accord ex-l coupe 2d with error HTTP Error 404: Not Found
Failed for honda, accord sport with error HTTP Error 404: Not Found
Failed for honda, accord ex-l with error HTTP Error 404: Not Found
Failed for honda, accord lx with error HTTP Error 404: Not Found
Failed for honda, nan with error HTTP Error 404: Not Found

...

Failed for ferrari, 488 gtb with error HTTP Error 404: Not Found
Failed for ferrari, 456gt with error HTTP Error 404: Not Found
Failed for ferrari, california with error HTTP Error 404: Not Found
Failed for ferrari, mondial with error HTTP Error 404: Not Found
Failed for ferrari, 430 f1 with error HTTP Error 404: Not Found
Failed for ferrari, spider with error HTTP Error 404: Not Found
Failed for ferrari, f430 spider f1 with error HTTP Error 404: Not Found
Failed for ferrari, 360 spider with error HTTP Error 404: Not Found
Failed for ferrari, ff coupe with error HTTP Error 404: Not Found
Failed for ferrari, mondial convertible with error HTTP Error 404: Not Found
Failed for ferrari, f355 spider 30k miles with error HTTP Error 404: Not Found
Failed for ferrari, 360 with error HTTP Error 404: Not Found
Failed for ferrari, 458 spider with error HTTP Error 404: Not Found
Failed for ferrari, 348 ts with error HTTP Error 404: Not Found
Failed for ferrari, f430 spider with error HTTP Error 40

Failed for datsun, 280zx with error HTTP Error 404: Not Found
Failed for datsun, 510 with error HTTP Error 404: Not Found
Failed for datsun, 300zx with error HTTP Error 404: Not Found
Failed for datsun, 280zx turbo with error HTTP Error 404: Not Found
Failed for datsun, 280 zx with error HTTP Error 404: Not Found



Failed for porche, boxster with error HTTP Error 404: Not Found
Failed for porche, cayenne with error HTTP Error 404: Not Found
Failed for porche, cayenne turbo awd with error HTTP Error 404: Not Found
Failed for porche, cayenne awd triptronic with error HTTP Error 404: Not Found
Failed for porche, panamera s with error HTTP Error 404: Not Found
Failed for porche, 911 carrera 4 with error HTTP Error 404: Not Found



Failed for land rover, discovery 11 with error HTTP Error 404: Not Found
Failed for land rover, lr3 hse 4wd with error HTTP Error 404: Not Found
Failed for land rover, defender with error HTTP Error 404: Not Found
Failed for land rover, nan with error HTTP Error 404: Not Found
Failed for land rover, rangerover hse with error HTTP Error 404: Not Found
Failed for land rover, sport hse with error HTTP Error 404: Not Found
Failed for land rover, freelander with error HTTP Error 404: Not Found
Failed for land rover, lr2 with error HTTP Error 404: Not Found
Failed for land rover, lr4 with error HTTP Error 404: Not Found



Failed for hennessey, corvette hpe700 with error HTTP Error 404: Not Found




Finally we save the results to disk so they may be accessed later without rerunning the scraping script. We save them in dictionary format as Python can only read the built-in types from txt file.

In [9]:
with open(Path("../data/similar_cars.txt", "w+") as f:
    f.write(str(dict(similar_cars)))